# Tutorial [Basic]

The goal of the package is to allow a quick and flexible way of constructing a table representing an accelerator lattice. This table should be used as input in the package latticeadaptor for generating accelerator lattice files in various formats and apply more advanced edits if necessary.

In [1]:
from latticeconstructor.core import LatticeBuilderLine
lblfodo = LatticeBuilderLine()

## Class properties

In [2]:
lblfodo.__dict__

{'lattice': [],
 'definitions': {},
 'table': None,
 'history': <queue.LifoQueue at 0x7f64f0356a60>}

## Class methods

In [3]:
from types import FunctionType
[x for x, y in LatticeBuilderLine.__dict__.items() if (type(y) == FunctionType) and not x.startswith('_')]

['add_def',
 'add_element',
 'replace_element',
 'replace_list',
 'insert_element_before',
 'insert_element_after',
 'remove_element',
 'remove_from_to',
 'get_idx',
 'build_table',
 'load_from_file',
 'load_lattice_from_file',
 'load_definitions_from_file',
 'undo']

## Example 1: FODO

In [4]:


# Element definitions
lblfodo.add_def(
        {
            "QF": {"family" : "KQUAD", "L": 0.342, "K1":  0.4900, "N_KICKS": 16},
            "QD": {"family" : "KQUAD", "L": 0.668, "K1": -0.4999, "N_KICKS": 16},
            "D":  {"family" : "DRIF" , "L": 3.5805},
            "W1": {"family" : "WATCH", "L": 0, "filename":"\"%s-%03ld.w1\"","mode": "coordinates"}
        }
)

# add elements in order
lblfodo.add_element(["W1","QF","D","QD","D","QF"])

# show lattice table
lblfodo.table

,family,L,filename,mode,name,K1,N_KICKS,pos
0,MARKER,0.0000,"""%s-%03ld.w1""",coordinates,W1,NaN,NaN,0.00000
1,QUADRUPOLE,0.3420,NaN,NaN,QF,0.4900,16.0,0.17100
2,DRIFT,3.5805,NaN,NaN,D,NaN,NaN,2.13225
3,QUADRUPOLE,0.6680,NaN,NaN,QD,-0.4999,16.0,4.25650
4,DRIFT,3.5805,NaN,NaN,D,NaN,NaN,6.38075
5,QUADRUPOLE,0.3420,NaN,NaN,QF,0.4900,16.0,8.34200


In [5]:
# check the defintions
lblfodo.definitions

{'QF': {'family': 'QUADRUPOLE',
  'L': 0.342,
  'K1': 0.49,
  'N_KICKS': 16,
  'name': 'QF'},
 'QD': {'family': 'QUADRUPOLE',
  'L': 0.668,
  'K1': -0.4999,
  'N_KICKS': 16,
  'name': 'QD'},
 'D': {'family': 'DRIFT', 'L': 3.5805, 'name': 'D'},
 'W1': {'family': 'MARKER',
  'L': 0,
  'filename': '"%s-%03ld.w1"',
  'mode': 'coordinates',
  'name': 'W1'}}

In [6]:
# check the lattice line definition (expanded)
lblfodo.lattice

['W1', 'QF', 'D', 'QD', 'D', 'QF']

## Example 2: Adding definitions and elements

Adding an element that is not defined will add it to the lattice but will not auto-update the table.

In [7]:
lblfodo.add_element("END")
lblfodo.lattice

Table not updated - not all elements defined.
{'END'}


['W1', 'QF', 'D', 'QD', 'D', 'QF', 'END']

In [8]:
lblfodo.table

,family,L,filename,mode,name,K1,N_KICKS,pos
0,MARKER,0.0000,"""%s-%03ld.w1""",coordinates,W1,NaN,NaN,0.00000
1,QUADRUPOLE,0.3420,NaN,NaN,QF,0.4900,16.0,0.17100
2,DRIFT,3.5805,NaN,NaN,D,NaN,NaN,2.13225
3,QUADRUPOLE,0.6680,NaN,NaN,QD,-0.4999,16.0,4.25650
4,DRIFT,3.5805,NaN,NaN,D,NaN,NaN,6.38075
5,QUADRUPOLE,0.3420,NaN,NaN,QF,0.4900,16.0,8.34200


To fix this, add the defintion and manually update the table.

In [9]:
# adding a marker
lblfodo.add_def({"END": {"family": "MARK", "L": 0.0 } })

# build table manually 
lblfodo.build_table()

In [10]:
lblfodo.lattice

['W1', 'QF', 'D', 'QD', 'D', 'QF', 'END']

In [11]:
# we highlight the change
lblfodo.table.style.apply(lambda x: ['background: lightgreen' if x.name in [6] else '' for i in x], axis=1)

,family,L,filename,mode,name,K1,N_KICKS,pos
0,MARKER,0.000000,"""%s-%03ld.w1""",coordinates,W1,nan,nan,0.000000
1,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,0.171000
2,DRIFT,3.580500,nan,nan,D,nan,nan,2.132250
3,QUADRUPOLE,0.668000,nan,nan,QD,-0.499900,16.000000,4.256500
4,DRIFT,3.580500,nan,nan,D,nan,nan,6.380750
5,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,8.342000
6,MARKER,0.000000,nan,nan,END,nan,nan,8.513000


## Example 3: Replacing elements

### Replace single element

In [12]:
lblfodo.replace_element('END','W1')

In [13]:
lblfodo.lattice

['W1', 'QF', 'D', 'QD', 'D', 'QF', 'W1']

In [14]:
# we again highlight the change
lblfodo.table.style.apply(lambda x: ['background: orange' if x.name in [6] else '' for i in x], axis=1)

,family,L,filename,mode,name,K1,N_KICKS,pos
0,MARKER,0.000000,"""%s-%03ld.w1""",coordinates,W1,nan,nan,0.000000
1,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,0.171000
2,DRIFT,3.580500,nan,nan,D,nan,nan,2.132250
3,QUADRUPOLE,0.668000,nan,nan,QD,-0.499900,16.000000,4.256500
4,DRIFT,3.580500,nan,nan,D,nan,nan,6.380750
5,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,8.342000
6,MARKER,0.000000,"""%s-%03ld.w1""",coordinates,W1,nan,nan,8.513000


In [15]:
# undo to keep the previous table
lblfodo.undo()

In [16]:
# check if undo worked
lblfodo.lattice

['W1', 'QF', 'D', 'QD', 'D', 'QF', 'END']

In [17]:
# check if undo worked
lblfodo.table.style.apply(lambda x: ['background: lightgreen' if x.name in [6] else '' for i in x], axis=1)

,family,L,filename,mode,name,K1,N_KICKS,pos
0,MARKER,0.000000,"""%s-%03ld.w1""",coordinates,W1,nan,nan,0.000000
1,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,0.171000
2,DRIFT,3.580500,nan,nan,D,nan,nan,2.132250
3,QUADRUPOLE,0.668000,nan,nan,QD,-0.499900,16.000000,4.256500
4,DRIFT,3.580500,nan,nan,D,nan,nan,6.380750
5,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,8.342000
6,MARKER,0.000000,nan,nan,END,nan,nan,8.513000


### Replace a series of elements

In [18]:
start_index = 5
stop_index  = 6
lblfodo.replace_list(start_index,stop_index,'QF')

In [19]:
lblfodo.lattice

['W1', 'QF', 'D', 'QD', 'D', 'QF']

In [20]:
lblfodo.table.style.apply(lambda x: ['background: orange' if x.name in [5] else '' for i in x], axis=1)

,family,L,filename,mode,name,K1,N_KICKS,pos
0,MARKER,0.000000,"""%s-%03ld.w1""",coordinates,W1,nan,nan,0.000000
1,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,0.171000
2,DRIFT,3.580500,nan,nan,D,nan,nan,2.132250
3,QUADRUPOLE,0.668000,nan,nan,QD,-0.499900,16.000000,4.256500
4,DRIFT,3.580500,nan,nan,D,nan,nan,6.380750
5,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,8.342000


## Example 4: Inserting elements

### Insert element before index

In [21]:
lblfodo.insert_element_before("END",5)

In [22]:
lblfodo.table.style.apply(lambda x: ['background: orange' if x.name in [5] else '' for i in x], axis=1)

,family,L,filename,mode,name,K1,N_KICKS,pos
0,MARKER,0.000000,"""%s-%03ld.w1""",coordinates,W1,nan,nan,0.000000
1,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,0.171000
2,DRIFT,3.580500,nan,nan,D,nan,nan,2.132250
3,QUADRUPOLE,0.668000,nan,nan,QD,-0.499900,16.000000,4.256500
4,DRIFT,3.580500,nan,nan,D,nan,nan,6.380750
5,MARKER,0.000000,nan,nan,END,nan,nan,8.171000
6,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,8.342000


In [23]:
# undo to keep the table
lblfodo.undo()

### Insert element after index

In [24]:
lblfodo.insert_element_after("END",5)

In [25]:
lblfodo.table.style.apply(lambda x: ['background: orange' if x.name in [6] else '' for i in x], axis=1)

,family,L,filename,mode,name,K1,N_KICKS,pos
0,MARKER,0.000000,"""%s-%03ld.w1""",coordinates,W1,nan,nan,0.000000
1,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,0.171000
2,DRIFT,3.580500,nan,nan,D,nan,nan,2.132250
3,QUADRUPOLE,0.668000,nan,nan,QD,-0.499900,16.000000,4.256500
4,DRIFT,3.580500,nan,nan,D,nan,nan,6.380750
5,QUADRUPOLE,0.342000,nan,nan,QF,0.490000,16.000000,8.342000
6,MARKER,0.000000,nan,nan,END,nan,nan,8.513000


In [26]:
lblfodo.lattice

['W1', 'QF', 'D', 'QD', 'D', 'QF', 'END']

In [27]:
lblfodo.undo()
lblfodo.lattice

['W1', 'QF', 'D', 'QD', 'D', 'QF']

## Example 5: Removing elements

### Remove at index

In [28]:
lblfodo.remove_element(2)

In [29]:
lblfodo.lattice

['W1', 'QF', 'QD', 'D', 'QF']

In [30]:
lblfodo.undo()
lblfodo.lattice

['W1', 'QF', 'D', 'QD', 'D', 'QF']

### Remove between indices

In [31]:
lblfodo.remove_from_to(2,4)
lblfodo.lattice

['W1', 'QF', 'QF']

In [32]:
lblfodo.undo()
lblfodo.lattice

['W1', 'QF', 'D', 'QD', 'D', 'QF']

## Example 6: getting int index of elements

In [33]:
lblfodo.get_idx('QF')

[1, 5]

# Tutorial [advanced]

The more advanced tutorial shows how to read in definitions and lattice constructions from various standard formats and prepare them to be manipulated by the LatticeBuilderLine class.

## Elegant

In [34]:
# generate lte lattice file 
elements ={
    "QF": {"type" : "KQUAD", "L": 0.342, "K1":  0.4900, "N_KICKS": 16},
    "QD": {"type" : "KQUAD", "L": 0.668, "K1": -0.4999, "N_KICKS": 16},
    "D":  {"type" : "DRIF" , "L": 3.5805},
    "W1": {"type" : "WATCH", "filename":"\"%s-%03ld.w1\"","mode": "coordinates"}
}

FODOstr    = "! FODO cell used for studying TRIBs\n\n"
stringlist = ["{:6}: {}".format(k,", ".join(["{}={:15.12f}".format(kk,vv) 
                                             if not isinstance(vv,str)
                                             else "{}={}".format(kk,vv)
                                             if kk!="type" else "{}".format(vv) for kk,vv in v.items()])) 
              for k,v in elements.items()]
line     = ["W1","QF","D","QD","D","QF"]
linestr  = "{:6}: LINE=({})".format("FODO",",".join(line))
FODOstr += "\n".join(stringlist)
FODOstr += "\n\n"
FODOstr += linestr

print(FODOstr)

with open("FODO_TRIB.lte","w") as f:
    f.write(FODOstr)

! FODO cell used for studying TRIBs

QF    : KQUAD, L= 0.342000000000, K1= 0.490000000000, N_KICKS=16.000000000000
QD    : KQUAD, L= 0.668000000000, K1=-0.499900000000, N_KICKS=16.000000000000
D     : DRIF, L= 3.580500000000
W1    : WATCH, filename="%s-%03ld.w1", mode=coordinates

FODO  : LINE=(W1,QF,D,QD,D,QF)


In [35]:
lbllte = LatticeBuilderLine()
lbllte.load_from_file("FODO_TRIB.lte",ftype='lte')

In [36]:
lbllte.lattice

['W1', 'QF', 'D', 'QD', 'D', 'QF']

In [37]:
lbllte.definitions

{'QF': {'family': 'QUADRUPOLE', 'L': 0.342, 'K1': 0.49, 'N_KICKS': 16.0},
 'QD': {'family': 'QUADRUPOLE', 'L': 0.668, 'K1': -0.4999, 'N_KICKS': 16.0},
 'D': {'family': 'DRIFT', 'L': 3.5805},
 'W1': {'family': 'MARKER', 'FILENAME': '%s-%03ld.w1', 'MODE': 'coordinates'}}

In [38]:
# the table needs to be updated manually
lbllte.table

In [39]:
lbllte.build_table()

In [40]:
lbllte.table

,family,FILENAME,MODE,L,K1,N_KICKS,pos
0,MARKER,%s-%03ld.w1,coordinates,NaN,NaN,NaN,NaN
1,QUADRUPOLE,NaN,NaN,0.3420,0.4900,16.0,0.17100
2,DRIFT,NaN,NaN,3.5805,NaN,NaN,2.13225
3,QUADRUPOLE,NaN,NaN,0.6680,-0.4999,16.0,4.25650
4,DRIFT,NaN,NaN,3.5805,NaN,NaN,6.38075
5,QUADRUPOLE,NaN,NaN,0.3420,0.4900,16.0,8.34200


<font color='red'>Remember to delete the columns that are attribute names not recognized by MADX before writing it to a sequence file.</font>

## MADX - Sequence file

In [41]:
FODOseqstr = """
QF : QUADRUPOLE, L := 0.50 , K1 :=  1.00;
QD : QUADRUPOLE, L := 1.00 , K1 := -1.00;
D1 : DRIFT, L := 1.00;
D2 : DRIFT, L := 1.00;

FODO: SEQUENCE, L=4.00;
QF, at = 0.25;
D1, at = 1.00;
QD, at = 2.00;
D2, at = 3.00;
QF, at = 3.75;
ENDSEQUENCE;
"""

print(FODOseqstr)

with open("FODO_TRIB.seq","w") as f:
    f.write(FODOseqstr)


QF : QUADRUPOLE, L := 0.50 , K1 :=  1.00;
QD : QUADRUPOLE, L := 1.00 , K1 := -1.00;
D1 : DRIFT, L := 1.00;
D2 : DRIFT, L := 1.00;

FODO: SEQUENCE, L=4.00;
QF, at = 0.25;
D1, at = 1.00;
QD, at = 2.00;
D2, at = 3.00;
QF, at = 3.75;
ENDSEQUENCE;



In [42]:
lblseq = LatticeBuilderLine()
lblseq.load_from_file("FODO_TRIB.seq",ftype='madx')
lblseq.lattice

['QF', 'D1', 'QD', 'D2']

In [43]:
lblseq.definitions

{'QF': {'family': 'QUADRUPOLE', 'L': 0.5, 'K1': 1.0},
 'QD': {'family': 'QUADRUPOLE', 'L': 1.0, 'K1': -1.0},
 'D1': {'family': 'DRIFT', 'L': 1.0},
 'D2': {'family': 'DRIFT', 'L': 1.0},
 'FODO': {'family': 'SEQUENCE', 'L': 4.0}}

In [44]:
lblseq.build_table()
lblseq.table

,family,L,K1,pos
0,QUADRUPOLE,0.5,1.0,0.25
1,DRIFT,1.0,NaN,1.00
2,QUADRUPOLE,1.0,-1.0,2.00
3,DRIFT,1.0,NaN,3.00


## MADX Line def

In [45]:
FODOlinestr = """
QF : QUADRUPOLE, L := 0.50 , K1 :=  1.00;
QD : QUADRUPOLE, L := 1.00 , K1 := -1.00;
D1 : DRIFT, L := 1.00;
D2 : DRIFT, L := 1.00;

FODO: LINE = (QF,D1,QD,D2,QF);
"""

print(FODOlinestr)

with open("FODO_TRIB.madx","w") as f:
    f.write(FODOlinestr)


QF : QUADRUPOLE, L := 0.50 , K1 :=  1.00;
QD : QUADRUPOLE, L := 1.00 , K1 := -1.00;
D1 : DRIFT, L := 1.00;
D2 : DRIFT, L := 1.00;

FODO: LINE = (QF,D1,QD,D2,QF);



In [46]:
lblmadx = LatticeBuilderLine()
lblmadx.load_from_file("FODO_TRIB.madx",ftype='madx')
lblmadx.lattice

['QF', 'D1', 'QD', 'D2', 'QF']

In [47]:
lblmadx.definitions

{'QF': {'family': 'QUADRUPOLE', 'L': 0.5, 'K1': 1.0},
 'QD': {'family': 'QUADRUPOLE', 'L': 1.0, 'K1': -1.0},
 'D1': {'family': 'DRIFT', 'L': 1.0},
 'D2': {'family': 'DRIFT', 'L': 1.0}}

In [48]:
lblmadx.build_table()
lblmadx.table

,family,L,K1,pos
0,QUADRUPOLE,0.5,1.0,0.25
1,DRIFT,1.0,NaN,1.00
2,QUADRUPOLE,1.0,-1.0,2.00
3,DRIFT,1.0,NaN,3.00
4,QUADRUPOLE,0.5,1.0,3.75
